In [63]:
import numpy as np
import ctypes as ct
import sys

In [58]:
class Cell:
    def __init__(self, KEY, PROB, V, U, W, CTU, STATE, DCU, CFL_DT, NEW_F, IK_F, DEL_F):
        self.key     = KEY
        self.prob    = PROB
        self.v       = V
        self.u       = U
        self.w       = W
        self.ctu     = CTU
        self.state   = STATE
        self.i_roots = None
        self.k_roots = None
        self.dcu     = DCU
        self.cfl_dt  = CFL_DT
        self.new_f   = NEW_F
        self.ik_f    = IK_F
        self.del_f   = DEL_F
        self.left    = None
        self.right   = None

class Grid:
    def __init__(self):
        self.thresh = None
        self.dt     = None
        self.epoch  = None
        self.dx     = None
        self.xh     = None

class Traj: 
    sigma = None
    b     = None
    r     = None
    T     = None


In [ ]:
class BST:
    def __init__(self):
        self.root = None
        self.dead = None
        self.a_count = 0
        self.tot_count = 1
        self.cfl_min_dt = sys.float_info.max


    def insert(self, cell):
        if self.root is None:
            self.root = cell
        else:
            self._insert_recursive(self.root, cell)

    def _insert_recursive(self, root, cell):
        if cell.key < root.key:
            if root.left is None:
                root.left = cell
            else:
                self._insert_recursive(root.left, cell)
        elif cell.key > root.key:
            if root.right is None:
                root.right = cell
            else:
                self._insert_recursive(root.right, cell)

    def search(self, cell):
        return self._search_recursive(self.root, cell)

    def _search_recursive(self, root, cell):
        if root is None or root.key == cell.key:
            return root
        if cell.key < root.key:
            return self._search_recursive(root.left, cell)
        else:
            return self._search_recursive(root.right, cell)

In [60]:
#========================================== Begin User Input ===============================================#
print( "Reading in user inputs...\n" )

DIM       = 3          # Dimensionality
NM        = 1          # Number of measurements 
FILE_PATH = "./Epochs" # Measurement file path

G           = Grid()   # Grid instance
G.thresh    = 2E-5     # Probability threshold
OUTPUT      = True     # Write info to terminal
RECORD      = True     # Write PDFs to .txt file
MEASURE     = True     # Take discrete measurement updates
OUTPUT_FREQ = 20       # Number of steps per output to terminal
DEL_STEP    = 25       # Number of steps per deletion procedure
NUM_DIST    = 6        # Number of distributions recorded per measurement
#===========================================================================================================#

Reading in user inputs...



In [61]:
#===================================== Read in measurement/trajectory info =================================#
print("Reading in discrete measurements...\n")

measurement_file = open(FILE_PATH + "/measurements.txt", "r")
mean = np.empty(DIM)
cov  = np.empty((DIM,DIM))

measurement_file.readline(); # Skip label line
mean = [float(string) for string in measurement_file.readline().split(" ")] # Mean
measurement_file.readline(); # Skip blank space
measurement_file.readline(); # Skip label line
for i in range(DIM):
    cov[i] = [float(string) for string in measurement_file.readline().split(" ")] # Covariance
G.epoch = mean
G.dx = [x/2 for x in np.diag(cov)] # Grid width
G.xh = [x/2 for x in G.dx]         # Half grid width

Lor = Traj() 
measurement_file.readline(); # Skip blank space
measurement_file.readline(); # Skip label line
Lor.sigma = float(measurement_file.readline())
measurement_file.readline(); # Skip label line
Lor.b = float(measurement_file.readline())
measurement_file.readline(); # Skip label line
Lor.r = float(measurement_file.readline())
measurement_file.readline(); # Skip label line
Lor.T = float(measurement_file.readline())

measure_time = Lor.T/NM                 # Time between measurements
record_time = measure_time/(NUM_DIST-1) # Time between recording PDF
measurement_file.close()
#===========================================================================================================#

Reading in discrete measurements...



In [62]:
print(Lor.sigma)

4.0
